In [ ]:
#pip install -r requirements.txt

In [1]:
import logging
import cvxpy as cvx
import numpy as np
import pickle
import pandas as pd

In [2]:
from doppelganger import (
    allocation,
    inputs,
    Configuration,
    HouseholdAllocator,
    PumsData,
    SegmentedData,
    BayesianNetworkModel,
    Population,
    Preprocessor,
    Marginals
)

In [3]:
from doppelganger.listbalancer import balance_multi_cvx, discretize_multi_weights

In [4]:
# deSerialization
with open('controls.pkl', 'rb') as file:
    controls = pickle.load(file)
with open('households_data.pkl', 'rb') as file:
    households_data = pickle.load(file)
with open('persons_data.pkl', 'rb') as file:
    persons_data = pickle.load(file)

In [5]:
controls.data

,Unnamed: 0,STATEFP,COUNTYFP,PUMA5CE,TRACTCE,num_people_count,num_people_1,num_people_3,num_people_2,num_people_4+,num_vehicles_1,num_vehicles_0,num_vehicles_2,num_vehicles_3+,age_0-17,age_18-34,age_65+,age_35-64
0,0,06,001,00106,430101,2217,305,356,648,908,270,45,1057,2076,1756,1161,671,3383
1,1,06,001,00106,430102,863,158,174,407,124,127,8,450,398,354,210,553,1022
2,2,06,001,00106,430200,2417,397,580,936,504,211,16,1380,1466,1310,911,1114,3324
3,3,06,001,00106,430300,1239,222,194,466,357,118,59,548,906,845,492,672,1597
4,4,06,001,00106,430400,752,136,150,294,172,181,6,308,661,311,331,416,997
5,5,06,001,00106,430500,2027,473,389,614,551,443,81,1283,873,1458,1264,582,2889
6,6,06,001,00106,430600,2145,496,477,699,473,425,0,1010,1421,988,1120,876,2755
7,7,06,001,00106,430700,1291,165,265,470,391,128,0,760,934,911,830,523,1649
8,8,06,001,00106,430800,2116,367,328,793,628,522,128,1335,909,1406,1377,896,2412
9,9,06,001,00106,430900,1822,499,356,589,378,724,40,893,467,1449,1186,587,1799


In [6]:
households_data.data

,puma,household_weight,state,num_people,household_income,num_vehicles,serial_number
0,00106,100,06,1,<=40000,0,1000481
1,00106,112,06,4+,40000+,3+,1002740
2,00106,78,06,3,40000+,2,1004323
3,00106,53,06,4+,40000+,3+,1006982
4,00106,63,06,2,40000+,2,1010099
5,00106,42,06,2,40000+,2,1011294
6,00106,255,06,0,<=40000,0,1011919
7,00106,72,06,4+,<=40000,2,1012613
8,00106,128,06,3,40000+,3+,1015561
9,00106,76,06,2,40000+,2,1016255


In [7]:
persons_data.data

,puma,age,state,individual_income,sex,person_weight,serial_number
0,00106,65+,06,<=0,M,100,1000481
1,00106,18-34,06,20000-40000,M,122,1002740
2,00106,18-34,06,0-20000,M,122,1002740
3,00106,65+,06,20000-40000,F,107,1002740
4,00106,65+,06,40000-80000,M,113,1002740
5,00106,18-34,06,<=0,M,76,1004323
6,00106,35-64,06,40000-80000,F,78,1004323
7,00106,35-64,06,20000-40000,M,68,1004323
8,00106,65+,06,<=0,M,53,1006982
9,00106,0-17,06,<=0,M,76,1006982


In [8]:
households, persons = HouseholdAllocator.from_cleaned_data0(controls, households_data, persons_data)

In [9]:
households

,serial_number,num_people,num_vehicles,household_weight,num_people_0,num_people_1,num_people_2,num_people_3,num_people_4+,num_vehicles_0,num_vehicles_1,num_vehicles_2,num_vehicles_3+,age_35-64,age_18-34,age_65+,age_0-17
618,4431,2,2,60,0,0,1,0,0,0,0,1,0,0,0,1,1
698,5379,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
219,12930,1,1,142,0,1,0,0,0,0,1,0,0,0,0,1,0
237,13176,3,2,200,0,0,0,1,0,0,0,1,0,1,0,2,0
393,17489,2,1,37,0,0,1,0,0,0,1,0,0,1,0,1,0
395,17679,1,1,39,0,1,0,0,0,0,1,0,0,0,0,0,1
398,17921,1,0,113,0,1,0,0,0,1,0,0,0,0,0,1,0
415,20249,4+,3+,79,0,0,0,0,1,0,0,0,1,1,1,2,0
419,21217,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
420,21307,2,2,63,0,0,1,0,0,0,0,1,0,0,0,0,2


In [10]:
persons

,serial_number,sex,age
0,1000481,M,65+
1,1002740,M,18-34
2,1002740,M,18-34
3,1002740,F,65+
4,1002740,M,65+
5,1004323,M,18-34
6,1004323,F,35-64
7,1004323,M,35-64
8,1006982,M,65+
9,1006982,M,0-17


In [11]:
# Only take nonzero weights
households = households[households[inputs.HOUSEHOLD_WEIGHT.name] > 0]
households

,serial_number,num_people,num_vehicles,household_weight,num_people_0,num_people_1,num_people_2,num_people_3,num_people_4+,num_vehicles_0,num_vehicles_1,num_vehicles_2,num_vehicles_3+,age_35-64,age_18-34,age_65+,age_0-17
618,4431,2,2,60,0,0,1,0,0,0,0,1,0,0,0,1,1
219,12930,1,1,142,0,1,0,0,0,0,1,0,0,0,0,1,0
237,13176,3,2,200,0,0,0,1,0,0,0,1,0,1,0,2,0
393,17489,2,1,37,0,0,1,0,0,0,1,0,0,1,0,1,0
395,17679,1,1,39,0,1,0,0,0,0,1,0,0,0,0,0,1
398,17921,1,0,113,0,1,0,0,0,1,0,0,0,0,0,1,0
415,20249,4+,3+,79,0,0,0,0,1,0,0,0,1,1,1,2,0
420,21307,2,2,63,0,0,1,0,0,0,0,1,0,0,0,0,2
421,21355,2,2,172,0,0,1,0,0,0,0,1,0,1,0,1,0
468,27462,1,1,64,0,1,0,0,0,0,1,0,0,0,0,0,1


In [12]:
households[inputs.HOUSEHOLD_WEIGHT.name]

618     60
219    142
237    200
393     37
395     39
398    113
415     79
420     63
421    172
468     64
469     81
471     62
475     40
482     47
493     93
497    109
501     65
504     64
514    102
524    180
546    101
554     79
568     86
573     74
584     58
605    227
607     76
628    145
635     46
639    117
      ... 
329    114
331     64
332    184
334     67
336     93
337    115
338    102
340     80
341     71
342    102
344     95
345     37
346     69
348     83
349    106
351    231
352     54
353     97
354     66
355     65
356    142
357    106
358    385
359    168
362    121
363    157
364     61
367    137
368     54
370    230
Name: household_weight, Length: 917, dtype: int64

In [13]:
households[inputs.HOUSEHOLD_WEIGHT.name].as_matrix()

G:\Anaconda\envs\dop2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([ 60, 142, 200,  37,  39, 113,  79,  63, 172,  64,  81,  62,  40,
        47,  93, 109,  65,  64, 102, 180, 101,  79,  86,  74,  58, 227,
        76, 145,  46, 117,  70, 208,  34, 101,  42,  51, 103, 174,  44,
        63,  64,  71,  70,  84,  55,  83,  58,  39,  49,  51,  70, 170,
        72,  75,  77,  49, 109,  87, 209,  67, 207, 223,  92,  94, 102,
        60, 139, 118,  55, 138, 118, 108, 193,  56, 154,  56,  37,  82,
       102, 160,  63, 177,  59,  67,  68,  60, 138,  87,  95,  37, 243,
        93,  66,  63,  94, 181,  58, 102, 101,  59,  52,  90, 334,  59,
        61, 182,  71,  41,  97,  42, 105,  66,  94, 131,  52, 336,  53,
        67, 228,  55,  53,  61,  45,  66,  84,  70, 186,  68, 128, 207,
        84, 168,  75,  86,  74,  74,  64, 125,  60, 121, 128,  71,  65,
        35,  78,  46, 167, 101, 113,  65,  50, 129,  48, 132,  79, 130,
        71,  34,  91,  63, 116,  76, 123,  85,  80, 325, 160,  42,  55,
        53,  69,  79, 166,  76,  41, 109,  67, 233,  53, 125, 11

In [14]:
hw_df = pd.DataFrame(households[inputs.HOUSEHOLD_WEIGHT.name])
hw_df.as_matrix().shape

G:\Anaconda\envs\dop2\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


(917, 1)

In [ ]:
hw_df.as_matrix().T.shape

In [ ]:

a = [1,2,3,4,5,6,7]
ps = pd.Series(a)
a_np = ps.to_numpy()
print(a_np.shape)
a_np

In [ ]:
a_np.T
print(a_np.T.shape)

In [233]:
# Initial weights from PUMS
w = households[inputs.HOUSEHOLD_WEIGHT.name].as_matrix().T######################may not be right, use to_numpy() and transpose instead maybe
print(w.shape)
w

(917,)


G:\Anaconda\envs\dop2\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


array([ 60, 142, 200,  37,  39, 113,  79,  63, 172,  64,  81,  62,  40,
        47,  93, 109,  65,  64, 102, 180, 101,  79,  86,  74,  58, 227,
        76, 145,  46, 117,  70, 208,  34, 101,  42,  51, 103, 174,  44,
        63,  64,  71,  70,  84,  55,  83,  58,  39,  49,  51,  70, 170,
        72,  75,  77,  49, 109,  87, 209,  67, 207, 223,  92,  94, 102,
        60, 139, 118,  55, 138, 118, 108, 193,  56, 154,  56,  37,  82,
       102, 160,  63, 177,  59,  67,  68,  60, 138,  87,  95,  37, 243,
        93,  66,  63,  94, 181,  58, 102, 101,  59,  52,  90, 334,  59,
        61, 182,  71,  41,  97,  42, 105,  66,  94, 131,  52, 336,  53,
        67, 228,  55,  53,  61,  45,  66,  84,  70, 186,  68, 128, 207,
        84, 168,  75,  86,  74,  74,  64, 125,  60, 121, 128,  71,  65,
        35,  78,  46, 167, 101, 113,  65,  50, 129,  48, 132,  79, 130,
        71,  34,  91,  63, 116,  76, 123,  85,  80, 325, 160,  42,  55,
        53,  69,  79, 166,  76,  41, 109,  67, 233,  53, 125, 11

In [234]:
# w = pd.DataFrame(households[inputs.HOUSEHOLD_WEIGHT.name]).as_matrix().T
# print(w.shape)
# w

In [235]:
allocation_inputs = [inputs.NUM_PEOPLE, inputs.NUM_VEHICLES] 
allocation_inputs

In [236]:
# Prepend column name to bin name to prevent bin collision
hh_columns = []
for a_input in allocation_inputs:
    subset_values = households[a_input.name].unique().tolist()
    hh_columns += HouseholdAllocator._str_broadcast(a_input.name, subset_values)
hh_columns

['num_people_2',
 'num_people_1',
 'num_people_3',
 'num_people_4+',
 'num_vehicles_2',
 'num_vehicles_1',
 'num_vehicles_0',
 'num_vehicles_3+']

In [237]:
hh_columns = HouseholdAllocator._filter_sparse_columns(households, hh_columns)
hh_columns

['num_people_2',
 'num_people_1',
 'num_people_3',
 'num_people_4+',
 'num_vehicles_2',
 'num_vehicles_1',
 'num_vehicles_3+']

In [238]:
hh_table = households[hh_columns].as_matrix()
print(hh_table.shape)
hh_table

(917, 7)


G:\Anaconda\envs\dop2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[1, 0, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 1, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 1, 0]], dtype=uint8)

In [239]:
# hh_table = households[hh_columns].to_numpy()
# print(hh_table.shape)
# hh_table

In [240]:
A = controls.data[hh_columns].as_matrix()
n_tracts, n_controls = A.shape
print(A.shape)
n_samples = len(households.index.values)
n_samples

(29, 7)


G:\Anaconda\envs\dop2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


917

In [241]:
mu = np.mat([1] * n_controls)
mu

matrix([[1, 1, 1, 1, 1, 1, 1]])

In [242]:
w_extend = np.tile(w, (n_tracts, 1))
print("w_extend: " + str(w_extend.shape))
mu_extend = np.mat(np.tile(mu, (n_tracts, 1)))
print("mu_extend: " + str(mu_extend.shape))
B = np.mat(np.dot(np.ones((1, n_tracts)), A)[0])
print("B: " + str(B.shape))

w_extend: (29, 917)
mu_extend: (29, 7)
B: (1, 7)


In [243]:
gamma = 100.
meta_gamma = 100.

In [244]:
w = w_extend
mu = gamma * mu_extend.T

In [245]:
n_samples, n_controls = hh_table.shape

# Solver won't converge with zero marginals. Identify and remove.
zero_marginals = np.where(~A.any(axis=1))[0]
zero_weights = np.zeros((1, n_samples))

if zero_marginals.size:
    logging.info(
        '%i tract(s) with zero marginals encountered. '
        'Setting weights to zero'.format(zero_marginals.size)
    )

    # Need to remove problem tracts and add a row of zeros later
    A = np.delete(A, zero_marginals, axis=0)
    w = np.delete(w, zero_marginals, axis=0)
    mu = np.delete(mu, zero_marginals, axis=1)

n_tracts = w.shape[0]
print(n_tracts)

29


In [246]:
x = cvx.Variable(shape=(n_tracts, n_samples))
print(x.shape)

(29, 917)


In [247]:
# Relative weights of tracts
# (need to reshape for numpy broadcasting)
wa = (np.sum(A, axis=1) / np.sum(A)).reshape(-1, 1)
w_relative = (np.array(w) * np.array(wa))

# With relaxation factors
z = cvx.Variable(shape=(n_controls, n_tracts))
q = cvx.Variable(shape=(n_controls,1))
identity = np.ones((n_tracts, 1))

In [248]:
solved = False
meta_mu=1000.
verbose_solver=False

In [249]:
while not solved:
    objective = cvx.Maximize(
        cvx.sum(cvx.entr(x) + cvx.multiply(cvx.log(np.e * w_relative), x)) +
        cvx.sum(cvx.multiply(mu, cvx.entr(z) + cvx.multiply(cvx.log(np.e), z))) +
        cvx.sum(cvx.multiply(meta_mu, cvx.entr(q) + cvx.multiply(cvx.log(np.e), q)))
    )
    

    # Update constraints
    constraints = [
        x >= 0,
        z >= 0,
        q >= 0,
#         x @ hh_table == A @ z.T,
#         (A.T @ z) * identity == B.T @ q
        x @ hh_table == cvx.multiply(A, z.T),
        cvx.multiply(A.T, z) * identity == cvx.multiply(B.T, q)
    ]

    # Define the problem
    prob = cvx.Problem(objective, constraints)

    # Solve the problem with updated exception handling
    try:
        prob.solve(verbose=verbose_solver)
        solved = True

    except cvx.SolverError:
        if np.all(mu == 1):
            # We can't reduce mu any further
            break
        mu = np.where(mu > 10, mu - 10, 1)
        logging.info('Solver error encountered. Importance weights have been relaxed.')

G:\Anaconda\envs\dop2\lib\site-packages\cvxpy\expressions\expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 17 times so far.

  warnings.warn(msg, UserWarning)


In [250]:
if not np.any(x.value):
    logging.exception('Solution infeasible. Using initial weights.')

# If we didn't get a value return the initial weights
weights_out = x.value if np.any(x.value) else w_relative

# Insert zeros
if zero_marginals.size:
    # Due to numpy insert behavior, we need to differentiate between the
    # values that go into the middle of the array, and the values that get appended
    weights_out = _insert_append(weights_out, zero_marginals, zero_weights, axis=0)

In [251]:
weights_out

array([[2.61331992, 3.39133066, 7.93541969, ..., 3.27191761, 2.67272812,
        5.4930004 ],
       [0.9490626 , 1.05030074, 2.70686072, ..., 1.01331832, 0.54771681,
        1.70119132],
       [2.78535504, 3.13737673, 9.06945498, ..., 3.02690572, 2.34457854,
        5.08166656],
       ...,
       [1.78227581, 4.17835474, 6.53235319, ..., 4.03122957, 2.1635986 ,
        6.7677578 ],
       [1.01587747, 2.53654392, 3.47789199, ..., 2.44722899, 1.27557768,
        4.10848662],
       [1.06356355, 0.72423102, 3.68209371, ..., 0.69872993, 0.87486444,
        1.17305024]])

In [252]:
weights_out.shape

(29, 917)

In [ ]:
def balance_multi_cvx(hh_table, A, B, w, mu=1000., meta_mu=1000., verbose_solver=False):
    """Maximum Entropy allocation method for multiple balanced units

    Args:
        hh_table (numpy matrix): Table of households categorical data
        A (numpy matrix): Area marginals (controls)
        B (numpy matrix): Meta-marginals
        w (numpy array): Initial household allocation weights
        mu (float): Importance weights of marginals for accuracy of fit
        meta_mu (float): Importance weights of meta-marginals for accuracy of fit
        verbose_solver (boolean): Provide detailed solver info

    Returns:
        (numpy matrix, numpy matrix, numpy matrix): Household weights,
            relaxation factors, relaxation factors,
    """

    n_samples, n_controls = hh_table.shape

    # Solver won't converge with zero marginals. Identify and remove.
    zero_marginals = np.where(~A.any(axis=1))[0]
    zero_weights = np.zeros((1, n_samples))

    if zero_marginals.size:
        logging.info(
            '%i tract(s) with zero marginals encountered. '
            'Setting weights to zero'.format(zero_marginals.size)
        )

        # Need to remove problem tracts and add a row of zeros later
        A = np.delete(A, zero_marginals, axis=0)
        w = np.delete(w, zero_marginals, axis=0)
        mu = np.delete(mu, zero_marginals, axis=1)

    n_tracts = w.shape[0]
    x = cvx.Variable(n_tracts, n_samples)

    # Relative weights of tracts
    # (need to reshape for numpy broadcasting)
    wa = (np.sum(A, axis=1) / np.sum(A)).reshape(-1, 1)
    w_relative = (np.array(w) * np.array(wa))

    # With relaxation factors
    z = cvx.Variable(n_controls, n_tracts)
    q = cvx.Variable(n_controls)

    identity = np.ones((n_tracts, 1))

    solved = False
    while not solved:
        objective = cvx.Maximize(
            cvx.sum_entries(cvx.entr(x) + cvx.mul_elemwise(cvx.log(np.e * w_relative), x)) +
            cvx.sum_entries(cvx.mul_elemwise(mu, cvx.entr(z) + cvx.mul_elemwise(cvx.log(np.e), z))) +
            cvx.sum_entries(cvx.mul_elemwise(meta_mu, cvx.entr(q) + cvx.mul_elemwise(cvx.log(np.e), q)))
        )

        constraints = [
            x >= 0,
            z >= 0,
            q >= 0,
            x * hh_table == cvx.mul_elemwise(A, z.T),
            cvx.mul_elemwise(A.T, z) * identity == cvx.mul_elemwise(B.T, q)
        ]

        prob = cvx.Problem(objective, constraints)

        try:
            prob.solve(verbose=verbose_solver)
            solved = True

        except cvx.SolverError:
            if np.all(mu == 1):
                # We can't reduce mu any further
                break
            mu = np.where(mu > 10, mu - 10, 1)
            logging.info('Solver error encountered. Importance weights have been relaxed.')

    if not np.any(x.value):
        logging.exception('Solution infeasible. Using initial weights.')

    # If we didn't get a value return the initial weights
    weights_out = x.value if np.any(x.value) else w_relative

    # Insert zeros
    if zero_marginals.size:
        # Due to numpy insert behavior, we need to differentiate between the
        # values that go into the middle of the array, and the values that get appended
        weights_out = _insert_append(
            weights_out, zero_marginals, zero_weights, axis=0)

    return weights_out

In [ ]:
# Household table (hh_table): Each column represents a type of household,
# and each row represents an area.
hh_table = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

# Area marginals (A): Controls for the number of each type of household in each area.
A = np.array([
    [50],  # Area 1 wants 50 of type 1 households
    [40],  # Area 2 wants 40 of type 2 households
    [30]   # Area 3 wants 30 of type 3 households
])

# Meta-marginals (B): Additional constraints across areas.
B = np.array([
    [60],  # Sum of types 1 and 2 households across all areas
    [40],  # Sum of types 2 and 3 households across all areas
    [120]  # Total households across all types and areas
])

# Initial weights (w): Start with equal weights for all households in all areas.
w = np.array([1, 1, 1])

# Importance weights for accuracy of fit (mu for A and meta_mu for B)
mu = 1000.
meta_mu = 1000.

# Verbose solver output for debugging
verbose_solver = True

# Call the function with the specified data
weights_out = balance_multi_cvx(hh_table, A, B, w, mu, meta_mu, verbose_solver)

# Print the output weights
print("Optimized Household Weights:")
print(weights_out)